# Tutorial 6: Statistical Mechanics with MechanicsDSL

This tutorial explores statistical mechanics concepts: ensembles, partition functions,
and the connection between microscopic physics and thermodynamic properties.

In [ ]:
from mechanics_dsl import PhysicsCompiler
from mechanics_dsl.domains.statistical import (
    CanonicalEnsemble, GrandCanonicalEnsemble, 
    IdealGas, QuantumOscillator, IsingModel
)
import matplotlib.pyplot as plt
import numpy as np

# Physical constants
k_B = 8.617e-5  # Boltzmann constant in eV/K

## 1. The Canonical Ensemble

A system in thermal equilibrium with a heat bath at temperature $T$.

### Key Concepts

- **Partition function**: $Z = \sum_i e^{-\beta E_i}$ where $\beta = 1/k_B T$
- **Free energy**: $F = -k_B T \ln Z$
- **Average energy**: $\langle E \rangle = -\frac{\partial \ln Z}{\partial \beta}$
- **Entropy**: $S = k_B (\ln Z + \beta \langle E \rangle)$

In [ ]:
# Two-level system (simplest example)
E0 = 0.0     # Ground state energy (eV)
E1 = 0.1     # Excited state energy (eV)

ensemble = CanonicalEnsemble(energy_levels=[E0, E1])

temperatures = np.linspace(10, 1000, 100)  # Kelvin
avg_energies = [ensemble.average_energy(T) for T in temperatures]
heat_capacities = [ensemble.heat_capacity(T) for T in temperatures]
entropies = [ensemble.entropy(T) for T in temperatures]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(temperatures, avg_energies, 'b-', lw=2)
axes[0].axhline(y=E1/2, color='r', linestyle='--', alpha=0.5, label='High-T limit')
axes[0].set_xlabel('Temperature (K)')
axes[0].set_ylabel('⟨E⟩ (eV)')
axes[0].set_title('Average Energy')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

axes[1].plot(temperatures, heat_capacities, 'g-', lw=2)
axes[1].set_xlabel('Temperature (K)')
axes[1].set_ylabel('C / k_B')
axes[1].set_title('Heat Capacity')
axes[1].grid(True, alpha=0.3)

axes[2].plot(temperatures, entropies, 'r-', lw=2)
axes[2].axhline(y=np.log(2), color='gray', linestyle='--', alpha=0.5, label='ln(2)')
axes[2].set_xlabel('Temperature (K)')
axes[2].set_ylabel('S / k_B')
axes[2].set_title('Entropy')
axes[2].grid(True, alpha=0.3)
axes[2].legend()

plt.suptitle('Two-Level System Thermodynamics', fontsize=14)
plt.tight_layout()
plt.show()

## 2. Quantum Harmonic Oscillator

Energy levels: $E_n = \hbar\omega(n + 1/2)$

### Partition Function

$$Z = \sum_{n=0}^{\infty} e^{-\beta\hbar\omega(n+1/2)} = \frac{e^{-\beta\hbar\omega/2}}{1 - e^{-\beta\hbar\omega}} = \frac{1}{2\sinh(\beta\hbar\omega/2)}$$

In [ ]:
# Quantum harmonic oscillator
hbar_omega = 0.05  # eV (typical vibrational energy)

qho = QuantumOscillator(hbar_omega=hbar_omega)

temperatures = np.linspace(10, 2000, 200)
avg_n = [qho.average_occupation(T) for T in temperatures]
avg_E = [qho.average_energy(T) for T in temperatures]
C = [qho.heat_capacity(T) for T in temperatures]

# Characteristic temperature
T_char = hbar_omega / k_B
print(f"Characteristic temperature: Θ = ℏω/k_B = {T_char:.1f} K")

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Average occupation number
axes[0].plot(temperatures, avg_n, 'b-', lw=2)
axes[0].axvline(x=T_char, color='r', linestyle='--', alpha=0.5, label=f'Θ = {T_char:.0f} K')
axes[0].set_xlabel('Temperature (K)')
axes[0].set_ylabel('⟨n⟩')
axes[0].set_title('Average Occupation Number')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Average energy
axes[1].plot(temperatures, avg_E, 'g-', lw=2, label='Quantum')
axes[1].plot(temperatures, [k_B * T for T in temperatures], 'r--', lw=2, alpha=0.5, label='Classical')
axes[1].set_xlabel('Temperature (K)')
axes[1].set_ylabel('⟨E⟩ (eV)')
axes[1].set_title('Average Energy')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

# Heat capacity
axes[2].plot(temperatures, C, 'purple', lw=2)
axes[2].axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Classical: C = k_B')
axes[2].axvline(x=T_char, color='orange', linestyle='--', alpha=0.5)
axes[2].set_xlabel('Temperature (K)')
axes[2].set_ylabel('C / k_B')
axes[2].set_title('Heat Capacity (Einstein Model)')
axes[2].grid(True, alpha=0.3)
axes[2].legend()

plt.suptitle(f'Quantum Harmonic Oscillator (ℏω = {hbar_omega} eV)', fontsize=14)
plt.tight_layout()
plt.show()

## 3. Ideal Gas

The classical ideal gas: $N$ non-interacting particles in volume $V$.

### Equations of State

- $PV = Nk_B T$ (ideal gas law)
- $\langle E \rangle = \frac{3}{2} Nk_B T$ (equipartition)
- $C_V = \frac{3}{2} Nk_B$

In [ ]:
# Ideal gas
N = 6.022e23  # Avogadro's number (1 mole)
V = 0.0224    # Volume in m³ (22.4 L at STP)

gas = IdealGas(N=N, V=V)

temperatures = np.linspace(100, 500, 100)
pressures = [gas.pressure(T) for T in temperatures]
energies = [gas.internal_energy(T) for T in temperatures]
entropies = [gas.entropy(T) for T in temperatures]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Pressure vs Temperature
axes[0].plot(temperatures, np.array(pressures)/1e5, 'b-', lw=2)
axes[0].set_xlabel('Temperature (K)')
axes[0].set_ylabel('Pressure (bar)')
axes[0].set_title('Equation of State: P vs T')
axes[0].grid(True, alpha=0.3)

# Internal energy
axes[1].plot(temperatures, np.array(energies)/1000, 'g-', lw=2)
axes[1].set_xlabel('Temperature (K)')
axes[1].set_ylabel('U (kJ)')
axes[1].set_title('Internal Energy')
axes[1].grid(True, alpha=0.3)

# Entropy (relative)
S_ref = entropies[0]
axes[2].plot(temperatures, np.array(entropies) - S_ref, 'r-', lw=2)
axes[2].set_xlabel('Temperature (K)')
axes[2].set_ylabel('ΔS (J/K)')
axes[2].set_title('Entropy Change')
axes[2].grid(True, alpha=0.3)

plt.suptitle('Ideal Gas Thermodynamics (1 mole at 22.4 L)', fontsize=14)
plt.tight_layout()
plt.show()

# Verify ideal gas law
T = 273.15  # K (STP)
P = gas.pressure(T)
print(f"At T = {T} K, V = {V*1000:.1f} L:")
print(f"Pressure P = {P/1e5:.3f} bar (should be ≈ 1 bar)")

## 4. Fermi-Dirac and Bose-Einstein Distributions

Quantum statistics for indistinguishable particles:

- **Fermions** (electrons, protons): $\langle n \rangle = \frac{1}{e^{\beta(E-\mu)} + 1}$
- **Bosons** (photons, phonons): $\langle n \rangle = \frac{1}{e^{\beta(E-\mu)} - 1}$

In [ ]:
from mechanics_dsl.domains.statistical import FermiDirac, BoseEinstein

mu = 0.0  # Chemical potential (eV)

# Compare distributions at different temperatures
energies = np.linspace(-0.3, 0.3, 200)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Fermi-Dirac at different temperatures
for T in [50, 100, 300, 1000]:
    fd = FermiDirac(T=T, mu=mu)
    occupation = [fd.occupation(E) for E in energies]
    axes[0].plot(energies, occupation, lw=2, label=f'T = {T} K')

axes[0].axvline(x=mu, color='k', linestyle='--', alpha=0.5)
axes[0].axhline(y=0.5, color='gray', linestyle=':', alpha=0.5)
axes[0].set_xlabel('Energy - μ (eV)')
axes[0].set_ylabel('⟨n⟩')
axes[0].set_title('Fermi-Dirac Distribution')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Bose-Einstein at different temperatures (positive energies only for μ=0)
energies_pos = np.linspace(0.01, 0.3, 200)
for T in [100, 300, 500, 1000]:
    be = BoseEinstein(T=T, mu=0)
    occupation = [be.occupation(E) for E in energies_pos]
    axes[1].plot(energies_pos, occupation, lw=2, label=f'T = {T} K')

axes[1].set_xlabel('Energy (eV)')
axes[1].set_ylabel('⟨n⟩')
axes[1].set_title('Bose-Einstein Distribution')
axes[1].set_ylim(0, 20)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. The Ising Model

A model of ferromagnetism: spins on a lattice interacting with neighbors.

### Hamiltonian

$$H = -J \sum_{\langle i,j \rangle} s_i s_j - h \sum_i s_i$$

where $s_i = \pm 1$, $J$ is the coupling, and $h$ is external field.

In [ ]:
# 1D Ising model (exact solution)
N = 100  # Number of spins
J = 0.01  # Coupling in eV

ising = IsingModel(N=N, J=J, dimension=1)

temperatures = np.linspace(1, 500, 100)
magnetizations = [ising.magnetization(T, h=0.001) for T in temperatures]
susceptibilities = [ising.susceptibility(T) for T in temperatures]
energies = [ising.internal_energy(T) for T in temperatures]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(temperatures, magnetizations, 'b-', lw=2)
axes[0].set_xlabel('Temperature (K)')
axes[0].set_ylabel('⟨m⟩')
axes[0].set_title('Magnetization (weak field h=0.001)')
axes[0].grid(True, alpha=0.3)

axes[1].plot(temperatures, susceptibilities, 'g-', lw=2)
axes[1].set_xlabel('Temperature (K)')
axes[1].set_ylabel('χ')
axes[1].set_title('Magnetic Susceptibility')
axes[1].grid(True, alpha=0.3)

axes[2].plot(temperatures, energies, 'r-', lw=2)
axes[2].set_xlabel('Temperature (K)')
axes[2].set_ylabel('⟨E⟩ / NJ')
axes[2].set_title('Internal Energy')
axes[2].grid(True, alpha=0.3)

plt.suptitle(f'1D Ising Model (N={N}, J={J} eV)', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Using the DSL for Statistical Mechanics

You can also define statistical systems using MechanicsDSL notation.

In [ ]:
dsl_code = r"""
\system{quantum_oscillator_ensemble}

\parameter{hbar_omega}{0.05}{eV}
\parameter{T}{300}{K}
\parameter{N}{1e23}{1}

\ensemble{canonical}
\hamiltonian{(n + 0.5) * hbar_omega}

% The system automatically computes:
% - Partition function Z
% - Average energy <E>
% - Heat capacity C
% - Entropy S
% - Free energy F
"""

compiler = PhysicsCompiler()
result = compiler.compile_dsl(dsl_code)

if result['success']:
    print("Statistical mechanics system compiled!")
    thermo = result.get('thermodynamic_quantities', {})
    for name, value in thermo.items():
        print(f"  {name}: {value}")

## Summary

In this tutorial, we explored:

1. **Canonical Ensemble** — Partition functions and thermodynamic quantities
2. **Quantum Harmonic Oscillator** — Einstein model of heat capacity
3. **Ideal Gas** — Equations of state and the ideal gas law
4. **Quantum Statistics** — Fermi-Dirac and Bose-Einstein distributions
5. **Ising Model** — Phase transitions and magnetism

## Key Takeaways

- **The partition function Z encodes everything** — All thermodynamic quantities follow from it
- **Quantum effects matter at low temperatures** — Heat capacity deviates from classical predictions
- **Phase transitions emerge from microscopic interactions** — The Ising model shows this beautifully

## Next Steps

Explore more physics domains with MechanicsDSL:
- Fluid dynamics (SPH)
- Electromagnetism
- Thermodynamic cycles